![](https://towardsml.files.wordpress.com/2019/09/bert.png)

I will be using
**sentence-transformers/msmarco-distilbert-base-v4**

### SBERT
- Sentence-Transformers is a state-of-the-art Python framework for embedding sentences, text and images. These embeddings can then be used for classification or clustering. For example, with the help of cosine similarity between embeddings, you can find sentences with similar meanings. This can be useful for semantic text similarity, semantic search, or paraphrase mining.

![](https://ichi.pro/assets/images/max/640/1*Bi-39aWOdFElYQOI7DSnWg.png)

To install Sentence Transformer use this command:

**!pip install -U sentence-transformers**

In [1]:
!pip install -U sentence-transformers

     |████████████████████████████████| 78 kB 689 kB/s            
  Preparing metadata (setup.py) ... - done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=87cde18262ab6562c37a28f4528113b2264d0bdaa095fe21c2409894cf28fde8
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers


In [2]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/msmarco-distilbert-base-v4')
embeddings = model.encode(sentences)
print(embeddings)

Downloading:   0%|          | 0.00/690 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.71k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[-0.2666365  -0.25132045 -0.86470926 ...  0.20419648 -0.7044446
  -0.26339564]
 [-0.340032    0.14204618 -0.46114454 ... -0.01900759  0.27845296
  -0.61292475]]


### Usage (HuggingFace Transformers)
- Without sentence-transformers, you can use the model like this: First, you pass your input through the transformer model, then you have to apply the right pooling-operation on-top of the contextualized word embeddings.



In [3]:
from transformers import AutoTokenizer, AutoModel
import torch


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Sentences we want sentence embeddings for
sentences = ['This is an example sentence', 'Each sentence is converted']

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4')
model = AutoModel.from_pretrained('sentence-transformers/msmarco-distilbert-base-v4')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Sentence embeddings:")
print(sentence_embeddings)

Downloading:   0%|          | 0.00/319 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/545 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/253M [00:00<?, ?B/s]

Sentence embeddings:
tensor([[-0.2666, -0.2513, -0.8647,  ...,  0.2042, -0.7044, -0.2634],
        [-0.3400,  0.1420, -0.4611,  ..., -0.0190,  0.2785, -0.6129]])


# Actual Impletation

- Since I have already install sentence transformer, I am commenting it

In [4]:
# !pip install sentence-transformers

- Importing library

In [5]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from sentence_transformers import SentenceTransformer

In [6]:
model = SentenceTransformer('msmarco-distilbert-base-v4', device="cuda")

In [7]:
df = pd.read_json("../input/corpusjson/corpus.json")

In [8]:
df.head()

,properties,url,file_id,type,language,file_time,import_date,update_date,file_data
0,WebResource,https://www.falconstor.com/blog/falconstor-vir...,f94daeb0-5f51-4c37-9613-29900482c140,webpage,eng,2019-09-16 17:09:42,2020-08-13T12:25:55.115835087,2020-08-13T12:25:55.115911142,FalconStor Virtual Tape Library (VTL) backup a...
1,WebResource,https://www.tripwire.com/state-of-security/fea...,e4cf5381-6a9e-436e-ad61-c1d4c256029d,webpage,eng,2020-01-18 18:17:52,2020-08-13T13:10:00.059732918,2020-08-13T13:10:00.059806426,More Yahoo users warned of malicious account a...
2,WebResource,https://www.nakivo.com/features/instant-file-r...,6f2e9c02-ec41-40ce-b411-3891c82e5e8d,webpage,eng,2020-01-27 05:20:42,2020-08-13T12:26:34.243574304,2020-08-13T12:26:34.243697528,File Recovery to Source & Custom Location for ...
3,WebResource,https://www.fortinet.com/cn/support/training/l...,851a24bd-7f66-4eef-a209-0a9f1dba7993,webpage,"eng,zho",2020-01-21 16:04:35,2020-08-13T12:55:19.441745432,2020-08-13T12:55:19.441829706,Authorized Training Centers (ATC)\n\nFortinet ...
4,WebResource,https://www.f5.com/company/blog/how-declarativ...,9383b15a-62d8-465c-9d6b-1ef524ebb12b,webpage,eng,2020-01-24 04:46:17,2020-08-13T13:10:39.207403014,2020-08-13T13:10:39.207521856,How Declarative Interfaces Democratize Infrast...


In [9]:
df.shape

(20000, 9)

In [10]:
df.columns

Index(['properties', 'url', 'file_id', 'type', 'language', 'file_time',
       'import_date', 'update_date', 'file_data'],
      dtype='object')

In [11]:
# Here we ancode all startup descriptions
# We do encoding in batches, as this reduces overhead costs and significantly speeds up the process
vectors = []
batch_size = 64
batch = []
for row in tqdm(df.itertuples()):
    description =row.file_data
    batch.append(description)
    if len(batch) >= batch_size:
        vectors.append(model.encode(batch))  # Text -> vector encoding happens here
        batch = []

if len(batch) > 0:
    vectors.append(model.encode(batch))
    batch = []

vectors = np.concatenate(vectors)

0it [00:00, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [12]:
# Now we have all our descriptions converted into vectors.
# We have 40474 vectors of 768 dimentions. The output layer of the model has this dimension
vectors.shape

(20000, 768)

In [13]:
# You can download this saved vectors and continue with rest part.
np.save("./final_vector_list.npy", vectors, allow_pickle=False)

In [14]:
# You can download this Model and continue with rest part.
model.save("./model")

- Evaluation 

In [15]:
from sklearn.metrics.pairwise import cosine_similarity

In [16]:
# Take a random description as a query
sample_query = df.iloc[12345].file_data
print(sample_query)

5 Best Software Tools for Managing Risk
Too often companies get stuck “making do” with spreadsheets and other outdated tools because, well, that’s the way it’s always been done. Having the right tool makes any job easier, and managing risk is no exception. New risk technology is challenging the status quo with tools that make managing risk smarter, faster and more strategic. Here are the five best software tools that will help you manage risk more effectively to drive organizational change, and ultimately lower your total cost of risk.
1. Incident Management & Root Cause Analysis Tools – Even one safety incident can be devastating to an organization’s financials and reputation. The key to preventing future problems is to get accurate information about every incident and near miss to those who can investigate, evaluate and take action. Technology takes incident reporting right to the source so data can be recorded while the event is still fresh. An incident reporting tool records data a

In [17]:
# Convert query description into a vector.
query_vector = model.encode(sample_query)  

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Look for the most similar vectors, manually score all vectors
scores = cosine_similarity([query_vector], vectors)[0] 

In [19]:
# Select top-5 with vectors the largest scores
top_scores_ids = np.argsort(scores)[-5:][::-1] 

In [20]:
# Displaying Selected top-5 vectors with the largest scores
top_scores_ids 

array([12345, 10776, 10757, 18790, 18472])

In [21]:
# Check if result similar to the query
for top_id in top_scores_ids:
    print("New_story: ")
    print(df.iloc[top_id].file_data)
    print("-------------------------------------------------------------------------------------")

New_story: 
5 Best Software Tools for Managing Risk
Too often companies get stuck “making do” with spreadsheets and other outdated tools because, well, that’s the way it’s always been done. Having the right tool makes any job easier, and managing risk is no exception. New risk technology is challenging the status quo with tools that make managing risk smarter, faster and more strategic. Here are the five best software tools that will help you manage risk more effectively to drive organizational change, and ultimately lower your total cost of risk.
1. Incident Management & Root Cause Analysis Tools – Even one safety incident can be devastating to an organization’s financials and reputation. The key to preventing future problems is to get accurate information about every incident and near miss to those who can investigate, evaluate and take action. Technology takes incident reporting right to the source so data can be recorded while the event is still fresh. An incident reporting tool re

# Conslusion:

- I think This performed very well on this dataset and I am also getting the desired output
- Since the transformer are huge in size its computationally expensive and takes a bit long time in training but performs wells!!